In [1]:
import sys

sys.path.append("..")
import glob

from zvar_utils.candidate import import_from_parquet, VariabilityCandidate
from zvar_utils.lightcurves import minify_lightcurve

/Users/theodlz/projects/ZVAR-Utilities/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path_candidates = "../data/variability_candidates"
path_lc = "../data/zvar/lc"
out_dir_lc = "../data/zvar/lc_minified"

In [3]:
bands = ["r", "g"]
fields = [648]
candidate_list: list[VariabilityCandidate] = []
for field in fields:
    for band in bands:
        for file in glob.glob(
            f"{path_candidates}/{field:04d}/*_z{band}.parquet", recursive=True
        ):
            candidate_list.extend(import_from_parquet(file))

# check that all the candidates have field, ccd, quad info
for candidate in candidate_list:
    if not candidate.field:
        print(candidate)
        raise ValueError("field not found")
    if not candidate.ccd:
        print(candidate)
        raise ValueError("ccd not found")
    if not candidate.quad:
        print(candidate)
        raise ValueError("quad not found")

# now groub them by field, ccd, quad
grouped_candidates: dict[tuple[int, int, int], list[VariabilityCandidate]] = {}
for candidate in candidate_list:
    key = (candidate.field, candidate.ccd, candidate.quad)
    if key not in grouped_candidates:
        grouped_candidates[key] = []
    grouped_candidates[key].append(candidate)

# now for each group, minify the associated lightcurve file(s)
for key, cands in grouped_candidates.items():
    field, ccd, quad = key
    psids = [c.psid for c in cands]
    if 144933586484299537 in psids:
        print(f"Field {field:04d}, CCD {ccd}, Quad {quad} has 144933586484299537")
        # check if 148250011324864469 is also in there
    if 148250011324864469 in psids:
        print(f"Field {field:04d}, CCD {ccd}, Quad {quad} has 148250011324864469")
    # check if 149280032410535655 is also in there
    if 149280032410535655 in psids:
        print(f"Field {field:04d}, CCD {ccd}, Quad {quad} has 149280032410535655")
    minify_lightcurve(
        field, ccd, quad, bands, psids, path_lc, out_dir_lc, delete_existing=True
    )

Field 0648, CCD 3, Quad 2 has 144933586484299537
Field 0648, CCD 10, Quad 3 has 148250011324864469
Field 0648, CCD 9, Quad 2 has 149280032410535655
